In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('../input/detecting-accounting-errors/Restate_int_train.csv')
test = pd.read_csv('../input/detecting-accounting-errors/Restate_int_test.csv')
external = pd.read_csv('../input/smu-forcasting/WRDS.csv')

In [3]:
train.head()

,gvkey,year,Filing,Date,Restate_Int
0,1004,2005,0001104659-05-033688,7/22/2005,0
1,1004,2006,0001104659-06-047248,7/17/2006,0
2,1004,2007,0001104659-07-055173,7/20/2007,0
3,1004,2008,0001047469-08-008126,7/11/2008,0
4,1004,2009,0001047469-09-006783,7/16/2009,0


In [4]:
test.head()

,gvkey,year,Filing,Date
0,1004,2010,0001047469-10-006500,7/16/2010
1,1045,2010,0000006201-10-000006,2/17/2010
2,1050,2010,0001193125-10-069639,3/29/2010
3,1072,2010,0000859163-10-000020,5/20/2010
4,1075,2010,0000950123-10-014366,2/19/2010


In [5]:
#change column name 
external.rename(columns={'GVKEY': 'gvkey', 'fyear': 'year'}, inplace=True)

In [6]:
external.head()

,gvkey,datadate,year,indfmt,consol,popsrc,datafmt,conm,curcd,acominc,act,aedi,am,ap,aqc,artfs,at,bkvlps,capx,ceq,ceqt,ch,che,ci,citotal,clt,cogs,cstk,dc,dlcch,dltt,dp,dpact,dptb,dptc,drc,drlt,dv,dvc,dvpd,...,ivpt,ivst,lcat,lct,lt,ni,opiti,optex,ppent,pstk,pvt,re,rect,revt,rveqt,rvti,seq,tstk,ui,urect,utfdoc,wcap,xacc,xlr,xpp,xt,costat,dvpsp_f,dvpsx_f,au,auop,auopic,conml,ggroup,gind,gsector,gsubind,sic,spcsrc,ipodate
0,1004,20050531,2004,INDL,C,D,STD,AAR CORP,USD,-19.779,474.542,NaN,NaN,77.015,0.000,NaN,732.230,9.6589,13.033,314.744,267.491,40.508,50.338,NaN,10.566,NaN,598.172,35.853,2.878,NaN,227.159,27.950,116.067,NaN,NaN,0.0,8.455,0.0,0.0,NaN,...,NaN,9.830,NaN,160.025,417.486,15.453,NaN,3.414,139.137,0.0,NaN,142.450,127.121,747.848,NaN,NaN,314.744,50.497,NaN,NaN,NaN,314.517,79.265,NaN,NaN,NaN,A,0.0,0.0,6.0,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
1,1004,20060531,2005,INDL,C,D,STD,AAR CORP,USD,-13.842,624.454,NaN,NaN,97.002,0.000,NaN,978.819,11.5326,16.296,422.717,375.928,121.738,121.738,NaN,41.100,NaN,704.081,40.789,5.956,NaN,318.576,29.222,129.896,NaN,NaN,0.0,7.994,0.0,0.0,NaN,...,NaN,0.000,NaN,187.788,556.102,35.163,NaN,3.080,213.380,0.0,NaN,183.550,136.272,897.284,NaN,NaN,422.717,69.664,NaN,NaN,NaN,436.666,88.497,NaN,NaN,NaN,A,0.0,0.0,6.0,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
2,1004,20070531,2006,INDL,C,D,STD,AAR CORP,USD,-13.899,645.721,NaN,NaN,110.239,38.478,NaN,1067.633,13.0998,29.891,494.243,418.105,83.317,83.317,NaN,62.140,NaN,837.171,42.230,5.327,NaN,253.611,32.199,145.518,NaN,NaN,0.0,7.533,0.0,0.0,NaN,...,NaN,0.000,NaN,256.506,573.390,58.660,NaN,2.038,260.167,0.0,NaN,242.153,181.691,1061.169,NaN,NaN,494.243,79.813,NaN,NaN,NaN,389.215,72.022,NaN,NaN,NaN,A,0.0,0.0,6.0,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
3,1004,20080531,2007,INDL,C,D,STD,AAR CORP,USD,-13.012,783.431,NaN,NaN,99.073,85.210,NaN,1362.010,15.0944,30.334,585.255,454.089,109.391,112.435,NaN,76.031,NaN,1080.895,43.932,10.294,NaN,507.918,39.952,166.070,NaN,NaN,0.0,7.071,0.0,0.0,NaN,...,NaN,3.044,NaN,218.499,776.755,75.144,NaN,1.271,310.393,0.0,NaN,318.184,202.472,1384.919,NaN,NaN,585.255,100.935,NaN,NaN,NaN,564.932,81.909,NaN,NaN,NaN,A,0.0,0.0,6.0,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0
4,1004,20090531,2008,INDL,C,D,STD,AAR CORP,USD,-23.996,851.312,NaN,4.852,100.651,0.000,NaN,1377.511,16.8937,27.535,656.895,505.900,112.505,112.505,NaN,67.667,NaN,1110.677,44.201,6.503,NaN,392.984,40.551,174.873,NaN,NaN,0.0,6.610,0.0,0.0,NaN,...,NaN,0.000,NaN,254.418,720.616,78.651,NaN,0.945,245.586,0.0,NaN,385.851,227.300,1423.976,NaN,NaN,656.895,103.159,NaN,NaN,NaN,596.894,83.685,NaN,NaN,NaN,A,0.0,0.0,6.0,1.0,1.0,AAR Corp,2010.0,201010.0,20.0,20101010.0,5080,B,19880101.0


In [7]:
def find_missing(data):
    cnt_missing = data.isnull().sum().values #number of missing values
    total = data.shape[0] # total records
    ratio_missing = cnt_missing/total*100 #percentage of missing
    
    return pd.DataFrame(data ={'missing_count': cnt_missing, 'missing_ratio_percentage' : ratio_missing}, index = data.columns.values)
df_missing = find_missing(external)
df_missing.style.set_properties(**{'text-align': 'left'})
df_missing.sort_values(['missing_count','missing_ratio_percentage'],ascending = False)

,missing_count,missing_ratio_percentage
aedi,35495,100.000000
artfs,35495,100.000000
clt,35495,100.000000
dptb,35495,100.000000
dptc,35495,100.000000
dvpd,35495,100.000000
fea,35495,100.000000
fel,35495,100.000000
iati,35495,100.000000
ip,35495,100.000000


In [8]:
df_missing.reset_index(drop= False,inplace = True)
ext_feature = df_missing[df_missing['missing_ratio_percentage']<15]['index'].values #take feature with less than 15% missing values 

In [9]:
#then we are using these feature to merge with our train and test data 
external = external[ext_feature] #filter out these feature
external = external.select_dtypes(exclude=['object']) #for now we dont take categorical features 
train = train.merge(right = external,on =['gvkey','year'],how= 'left')
test = test.merge(right = external,on =['gvkey','year'],how= 'left')

In [10]:
train.shape

(15213, 51)

In [11]:
#convert to date time
train['Date'] =  pd.to_datetime(train['Date'])
test['Date'] =  pd.to_datetime(test['Date'])

train.head()

,gvkey,year,Filing,Date,Restate_Int,datadate,acominc,ap,aqc,at,bkvlps,capx,ceq,ceqt,ch,che,cogs,cstk,dltt,dp,dv,dvc,dvt,ebit,ebitda,epsfi,epspi,gdwl,gp,intan,invt,ivst,lt,ni,ppent,pstk,re,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,ggroup,gind,gsector,gsubind,sic
0,1004,2005,0001104659-05-033688,2005-07-22,0,20060531.0,-13.842,97.002,0.000,978.819,11.5326,16.296,422.717,375.928,121.738,121.738,704.081,40.789,318.576,29.222,0.0,0.0,0.0,62.655,91.877,0.94,1.05,44.432,193.203,46.789,323.592,0.000,556.102,35.163,213.380,0.0,183.550,136.272,897.284,422.717,69.664,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0
1,1004,2006,0001104659-06-047248,2006-07-17,0,20070531.0,-13.899,110.239,38.478,1067.633,13.0998,29.891,494.243,418.105,83.317,83.317,837.171,42.230,253.611,32.199,0.0,0.0,0.0,86.708,118.907,1.40,1.61,72.687,223.998,76.138,342.593,0.000,573.390,58.660,260.167,0.0,242.153,181.691,1061.169,494.243,79.813,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0
2,1004,2007,0001104659-07-055173,2007-07-20,0,20080531.0,-13.012,99.073,85.210,1362.010,15.0944,30.334,585.255,454.089,109.391,112.435,1080.895,43.932,507.918,39.952,0.0,0.0,0.0,128.570,168.522,1.76,2.02,101.520,304.024,131.166,435.608,3.044,776.755,75.144,310.393,0.0,318.184,202.472,1384.919,585.255,100.935,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0
3,1004,2008,0001047469-08-008126,2008-07-11,0,20090531.0,-23.996,100.651,0.000,1377.511,16.8937,27.535,656.895,505.900,112.505,112.505,1110.677,44.201,392.984,40.551,0.0,0.0,0.0,125.529,166.080,1.87,2.07,109.751,313.299,150.995,477.424,0.000,720.616,78.651,245.586,0.0,385.851,227.300,1423.976,656.895,103.159,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0
4,1004,2009,0001047469-09-006783,2009-07-16,0,20100531.0,-29.646,114.906,193.989,1501.042,18.9167,28.855,746.906,577.478,79.370,79.370,1065.902,44.870,336.191,38.930,0.0,0.0,0.0,95.415,134.345,1.16,1.17,111.544,286.249,169.428,496.904,0.000,754.692,44.628,334.430,0.0,389.641,238.466,1352.151,746.906,104.447,0.0,0.0,6.0,4.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0


In [12]:
train['Restate_Int'].value_counts()

0    14850
1      363
Name: Restate_Int, dtype: int64

In [13]:
#sort by gvkey and year
train.sort_values(['gvkey','year'],inplace = True)
test.sort_values(['gvkey','year'],inplace = True)

In [14]:
train['year'].value_counts()

2005    3112
2009    3111
2006    3017
2008    2998
2007    2975
Name: year, dtype: int64

# Feature engineering

In [15]:
train['Weekday'] = train['Date'].dt.dayofweek
test['Weekday'] = test['Date'].dt.dayofweek
test.head()

,gvkey,year,Filing,Date,datadate,acominc,ap,aqc,at,bkvlps,capx,ceq,ceqt,ch,che,cogs,cstk,dltt,dp,dv,dvc,dvt,ebit,ebitda,epsfi,epspi,gdwl,gp,intan,invt,ivst,lt,ni,ppent,pstk,re,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,ggroup,gind,gsector,gsubind,sic,Weekday
0,1004,2010,0001047469-10-006500,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,1045,2010,0000006201-10-000006,2010-02-17,20101231.0,-2755.000,1156.000,0.0,25088.000,-11.8309,1962.000,-3945.000,-4877.000,168.000,4946.000,18138.000,339.000,9253.000,995.000,0.000,0.000,0.000,308.000,1303.000,-1.41,-1.41,0.000,4032.000,932.000,594.000,4778.0,29033.000,-471.000,15082.000,0.0,-8362.000,738.000,22170.000,-3945.000,367.000,0.0,0.0,4.0,1.0,1.0,2030.0,203020.0,20.0,20302010.0,4512.0,2
2,1050,2010,0001193125-10-069639,2010-03-29,20101231.0,-1.608,9.712,0.0,74.791,2.4565,0.654,35.174,16.270,5.792,5.792,106.692,0.144,10.800,1.758,0.000,0.000,0.000,4.640,6.398,0.15,0.15,14.713,33.910,18.904,12.777,0.0,39.617,2.105,5.880,0.0,-7.851,26.772,140.602,35.174,0.356,0.0,0.0,11.0,1.0,0.0,2020.0,202010.0,20.0,20201050.0,3564.0,0
3,1072,2010,0000859163-10-000020,2010-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,1075,2010,0000950123-10-014366,2010-02-19,20101231.0,-159.767,236.354,0.0,12362.703,33.8634,765.152,3683.327,3498.375,110.188,110.188,2125.206,2421.372,3045.794,414.555,216.979,224.305,224.305,723.884,1138.439,3.27,3.28,NaN,1138.439,184.952,202.989,0.0,8587.477,350.053,9393.867,0.0,1264.194,419.518,3263.645,3683.327,2.239,2.1,2.1,5.0,4.0,1.0,5510.0,551010.0,55.0,55101010.0,4911.0,4


In [16]:
train['Date_lag'] = train.groupby('gvkey')['Date'].shift(1)
test['Date_lag'] = test.groupby('gvkey')['Date'].shift(1)
train['Date_diff'] = (train['Date'] -train['Date_lag']).dt.days -365
test['Date_diff'] = (test['Date'] -test['Date_lag']).dt.days - 365

In [17]:
train.head()

,gvkey,year,Filing,Date,Restate_Int,datadate,acominc,ap,aqc,at,bkvlps,capx,ceq,ceqt,ch,che,cogs,cstk,dltt,dp,dv,dvc,dvt,ebit,ebitda,epsfi,epspi,gdwl,gp,intan,invt,ivst,lt,ni,ppent,pstk,re,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,ggroup,gind,gsector,gsubind,sic,Weekday,Date_lag,Date_diff
0,1004,2005,0001104659-05-033688,2005-07-22,0,20060531.0,-13.842,97.002,0.000,978.819,11.5326,16.296,422.717,375.928,121.738,121.738,704.081,40.789,318.576,29.222,0.0,0.0,0.0,62.655,91.877,0.94,1.05,44.432,193.203,46.789,323.592,0.000,556.102,35.163,213.380,0.0,183.550,136.272,897.284,422.717,69.664,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,NaT,NaN
1,1004,2006,0001104659-06-047248,2006-07-17,0,20070531.0,-13.899,110.239,38.478,1067.633,13.0998,29.891,494.243,418.105,83.317,83.317,837.171,42.230,253.611,32.199,0.0,0.0,0.0,86.708,118.907,1.40,1.61,72.687,223.998,76.138,342.593,0.000,573.390,58.660,260.167,0.0,242.153,181.691,1061.169,494.243,79.813,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,0,2005-07-22,-5.0
2,1004,2007,0001104659-07-055173,2007-07-20,0,20080531.0,-13.012,99.073,85.210,1362.010,15.0944,30.334,585.255,454.089,109.391,112.435,1080.895,43.932,507.918,39.952,0.0,0.0,0.0,128.570,168.522,1.76,2.02,101.520,304.024,131.166,435.608,3.044,776.755,75.144,310.393,0.0,318.184,202.472,1384.919,585.255,100.935,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,2006-07-17,3.0
3,1004,2008,0001047469-08-008126,2008-07-11,0,20090531.0,-23.996,100.651,0.000,1377.511,16.8937,27.535,656.895,505.900,112.505,112.505,1110.677,44.201,392.984,40.551,0.0,0.0,0.0,125.529,166.080,1.87,2.07,109.751,313.299,150.995,477.424,0.000,720.616,78.651,245.586,0.0,385.851,227.300,1423.976,656.895,103.159,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,2007-07-20,-8.0
4,1004,2009,0001047469-09-006783,2009-07-16,0,20100531.0,-29.646,114.906,193.989,1501.042,18.9167,28.855,746.906,577.478,79.370,79.370,1065.902,44.870,336.191,38.930,0.0,0.0,0.0,95.415,134.345,1.16,1.17,111.544,286.249,169.428,496.904,0.000,754.692,44.628,334.430,0.0,389.641,238.466,1352.151,746.906,104.447,0.0,0.0,6.0,4.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,3,2008-07-11,5.0


In [18]:
train['Week_num'] =train['Date'].dt.week#.dt.date
test['Week_num'] =test['Date'].dt.week#.dt.date
train.head()

,gvkey,year,Filing,Date,Restate_Int,datadate,acominc,ap,aqc,at,bkvlps,capx,ceq,ceqt,ch,che,cogs,cstk,dltt,dp,dv,dvc,dvt,ebit,ebitda,epsfi,epspi,gdwl,gp,intan,invt,ivst,lt,ni,ppent,pstk,re,rect,revt,seq,tstk,dvpsp_f,dvpsx_f,au,auop,auopic,ggroup,gind,gsector,gsubind,sic,Weekday,Date_lag,Date_diff,Week_num
0,1004,2005,0001104659-05-033688,2005-07-22,0,20060531.0,-13.842,97.002,0.000,978.819,11.5326,16.296,422.717,375.928,121.738,121.738,704.081,40.789,318.576,29.222,0.0,0.0,0.0,62.655,91.877,0.94,1.05,44.432,193.203,46.789,323.592,0.000,556.102,35.163,213.380,0.0,183.550,136.272,897.284,422.717,69.664,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,NaT,NaN,29
1,1004,2006,0001104659-06-047248,2006-07-17,0,20070531.0,-13.899,110.239,38.478,1067.633,13.0998,29.891,494.243,418.105,83.317,83.317,837.171,42.230,253.611,32.199,0.0,0.0,0.0,86.708,118.907,1.40,1.61,72.687,223.998,76.138,342.593,0.000,573.390,58.660,260.167,0.0,242.153,181.691,1061.169,494.243,79.813,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,0,2005-07-22,-5.0,29
2,1004,2007,0001104659-07-055173,2007-07-20,0,20080531.0,-13.012,99.073,85.210,1362.010,15.0944,30.334,585.255,454.089,109.391,112.435,1080.895,43.932,507.918,39.952,0.0,0.0,0.0,128.570,168.522,1.76,2.02,101.520,304.024,131.166,435.608,3.044,776.755,75.144,310.393,0.0,318.184,202.472,1384.919,585.255,100.935,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,2006-07-17,3.0,29
3,1004,2008,0001047469-08-008126,2008-07-11,0,20090531.0,-23.996,100.651,0.000,1377.511,16.8937,27.535,656.895,505.900,112.505,112.505,1110.677,44.201,392.984,40.551,0.0,0.0,0.0,125.529,166.080,1.87,2.07,109.751,313.299,150.995,477.424,0.000,720.616,78.651,245.586,0.0,385.851,227.300,1423.976,656.895,103.159,0.0,0.0,6.0,1.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,4,2007-07-20,-8.0,28
4,1004,2009,0001047469-09-006783,2009-07-16,0,20100531.0,-29.646,114.906,193.989,1501.042,18.9167,28.855,746.906,577.478,79.370,79.370,1065.902,44.870,336.191,38.930,0.0,0.0,0.0,95.415,134.345,1.16,1.17,111.544,286.249,169.428,496.904,0.000,754.692,44.628,334.430,0.0,389.641,238.466,1352.151,746.906,104.447,0.0,0.0,6.0,4.0,1.0,2010.0,201010.0,20.0,20101010.0,5080.0,3,2008-07-11,5.0,29


# LightGBM model

In [19]:
train.columns

Index(['gvkey', 'year', 'Filing', 'Date', 'Restate_Int', 'datadate', 'acominc',
       'ap', 'aqc', 'at', 'bkvlps', 'capx', 'ceq', 'ceqt', 'ch', 'che', 'cogs',
       'cstk', 'dltt', 'dp', 'dv', 'dvc', 'dvt', 'ebit', 'ebitda', 'epsfi',
       'epspi', 'gdwl', 'gp', 'intan', 'invt', 'ivst', 'lt', 'ni', 'ppent',
       'pstk', 're', 'rect', 'revt', 'seq', 'tstk', 'dvpsp_f', 'dvpsx_f', 'au',
       'auop', 'auopic', 'ggroup', 'gind', 'gsector', 'gsubind', 'sic',
       'Weekday', 'Date_lag', 'Date_diff', 'Week_num'],
      dtype='object')

In [20]:
train.select_dtypes(include=['float64']).columns

Index(['datadate', 'acominc', 'ap', 'aqc', 'at', 'bkvlps', 'capx', 'ceq',
       'ceqt', 'ch', 'che', 'cogs', 'cstk', 'dltt', 'dp', 'dv', 'dvc', 'dvt',
       'ebit', 'ebitda', 'epsfi', 'epspi', 'gdwl', 'gp', 'intan', 'invt',
       'ivst', 'lt', 'ni', 'ppent', 'pstk', 're', 'rect', 'revt', 'seq',
       'tstk', 'dvpsp_f', 'dvpsx_f', 'au', 'auop', 'auopic', 'ggroup', 'gind',
       'gsector', 'gsubind', 'sic', 'Date_diff'],
      dtype='object')

In [21]:

def create_new_col(aggs):
    return [k + "_"+ v for k in aggs for v in aggs[k]]
aggs1 = {'Weekday':['mean','max','min'],
#        'Date_diff':['max','min','std','mean'],
       'Week_num':['mean','max','min'],
       'Restate_Int':['max']}
aggs2 = {'Weekday':['mean','max','min'],
#        'Date_diff':['max','min','std','mean'],
       'Week_num':['mean','max','min']}
initial_feature = ['Weekday','Date_diff','Week_num','Restate_Int','gvkey','year','Date_lag','Date_diff','Filing']
total_feature = train.select_dtypes(include=['float64']).columns


for feature in total_feature:
    if feature not in initial_feature:
        print(feature)
        aggs1[feature] = ['mean',]
        aggs2[feature] = ['mean',]
print('done')



train_agg = train.groupby('gvkey').agg(aggs1)
train_agg.columns = create_new_col(aggs1)
train_agg.reset_index(inplace = True)
y = train_agg['Restate_Int_max']
del train_agg['Restate_Int_max']
train_agg.head()


test_agg = test.groupby('gvkey').agg(aggs2)
test_agg.columns = create_new_col(aggs2)
test_agg.reset_index(inplace = True)
# print(dtypes(y))

datadate
acominc
ap
aqc
at
bkvlps
capx
ceq
ceqt
ch
che
cogs
cstk
dltt
dp
dv
dvc
dvt
ebit
ebitda
epsfi
epspi
gdwl
gp
intan
invt
ivst
lt
ni
ppent
pstk
re
rect
revt
seq
tstk
dvpsp_f
dvpsx_f
au
auop
auopic
ggroup
gind
gsector
gsubind
sic
done


In [22]:
train_agg.fillna(0,inplace = True)

In [23]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
columns = train_agg.columns

train_agg, y = sm.fit_sample(train_agg, y)

train_agg = pd.DataFrame(data=train_agg, columns = columns)
y = pd.DataFrame(data=y)

Using TensorFlow backend.


In [24]:
train_agg.head()

,gvkey,Weekday_mean,Weekday_max,Weekday_min,Week_num_mean,Week_num_max,Week_num_min,datadate_mean,acominc_mean,ap_mean,aqc_mean,at_mean,bkvlps_mean,capx_mean,ceq_mean,ceqt_mean,ch_mean,che_mean,cogs_mean,cstk_mean,dltt_mean,dp_mean,dv_mean,dvc_mean,dvt_mean,ebit_mean,ebitda_mean,epsfi_mean,epspi_mean,gdwl_mean,gp_mean,intan_mean,invt_mean,ivst_mean,lt_mean,ni_mean,ppent_mean,pstk_mean,re_mean,rect_mean,revt_mean,seq_mean,tstk_mean,dvpsp_f_mean,dvpsx_f_mean,au_mean,auop_mean,auopic_mean,ggroup_mean,gind_mean,gsector_mean,gsubind_mean,sic_mean
0,1004.0,3.0,4.0,0.0,28.8,29.0,28.0,20080531.0,-18.879,104.3742,63.5354,1257.403,15.10744,26.5822,581.2032,466.300,101.2642,101.873,959.7452,43.2044,361.856,36.1708,0.000,0.000,0.000,99.7754,135.9462,1.426,1.584,87.9868,264.1546,114.9032,415.2242,0.6088,676.311,58.4492,272.7912,0.0,303.8758,197.2402,1223.8998,581.2032,91.6036,0.00,0.00,6.0,1.6,1.0,2010.0,201010.0,20.0,20101010.0,5080.0
1,1013.0,1.5,4.0,0.0,27.0,52.0,2.0,20066031.0,-11.800,89.3500,95.3000,1708.050,7.71945,34.6000,892.3000,475.625,351.0000,549.100,795.1000,23.4500,412.825,71.1500,0.000,0.000,0.000,89.0000,160.1500,0.505,0.515,269.1750,512.3250,416.6750,159.7250,198.1000,815.750,60.2000,200.9750,0.0,-542.4250,222.7750,1307.4250,892.3000,0.0000,0.00,0.00,4.0,2.5,1.0,4520.0,452010.0,45.0,45201020.0,3661.0
2,1021.0,3.0,4.0,1.0,39.0,41.0,37.0,0.0,0.000,0.0000,0.0000,0.000,0.00000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1034.0,3.0,3.0,3.0,13.0,13.0,13.0,20051231.0,47.852,35.3600,0.0000,1623.383,16.97690,38.9390,918.0780,625.248,800.0100,800.010,150.2570,10.8820,0.017,67.1060,9.481,9.481,9.481,95.9950,163.1010,2.520,2.550,116.7470,403.3600,292.8300,92.7410,0.0000,705.305,133.7690,215.1740,0.0,-175.2850,90.8980,553.6170,918.0780,7.6440,0.18,0.18,11.0,1.0,2.0,3520.0,352020.0,35.0,35202010.0,2834.0
4,1045.0,3.4,4.0,2.0,8.0,8.0,8.0,20071231.0,-1500.200,1069.8000,0.0000,27564.800,-4.23450,864.4000,-1170.2000,-2293.000,150.2000,4579.000,17936.8000,260.4000,11034.800,1095.0000,0.000,0.000,0.000,174.6000,1269.6000,-3.084,-2.998,0.0000,4034.0000,1122.8000,540.8000,4428.8000,28735.000,-733.0000,17018.4000,0.0,-4311.0000,917.0000,21970.8000,-1170.2000,449.4000,0.00,0.00,4.0,2.2,1.0,2030.0,203020.0,20.0,20302010.0,4512.0


In [25]:
y.dtypes

0    int64
dtype: object

In [26]:
#get number of feature and number of training rows
X_train = train_agg
X_test = test_agg

num_train, num_feature = X_train.shape

#Get parameters 
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 5,
    'learning_rate': 0.001,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1000,
}


#------------------------------------------------------------
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)


oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])
feature_importance_df = pd.DataFrame()
feature_name = [col for col in X_train.columns]
print(feature_name)
print('Starting training...')
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train,y)):
    print('fold {} '.format(n_fold))
    trn_x, trn_y = X_train[feature_name].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X_train[feature_name].iloc[val_idx], y.iloc[val_idx]
    lgb_train = lgb.Dataset(trn_x, trn_y)
    lgb_eval = lgb.Dataset(val_x, val_y)

    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5000,
                   verbose_eval=1000)
    
    oof_preds[val_idx] = gbm.predict(val_x, num_iteration=gbm.best_iteration) #get oof prediction
#     sub_preds += gbm.predict_proba(X_test[feature_name],num_iteration=gbm.best_iteration)[:, 1]/folds.n_splits 
    #predict on test set, take average
    sub_preds += gbm.predict(X_test[feature_name], num_iteration=gbm.best_iteration) / folds.n_splits 
    
    #save the feature important 
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feature_name
    fold_importance_df["importance"] = np.log1p(gbm.feature_importance(
        importance_type='gain',
        iteration=gbm.best_iteration))
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

['gvkey', 'Weekday_mean', 'Weekday_max', 'Weekday_min', 'Week_num_mean', 'Week_num_max', 'Week_num_min', 'datadate_mean', 'acominc_mean', 'ap_mean', 'aqc_mean', 'at_mean', 'bkvlps_mean', 'capx_mean', 'ceq_mean', 'ceqt_mean', 'ch_mean', 'che_mean', 'cogs_mean', 'cstk_mean', 'dltt_mean', 'dp_mean', 'dv_mean', 'dvc_mean', 'dvt_mean', 'ebit_mean', 'ebitda_mean', 'epsfi_mean', 'epspi_mean', 'gdwl_mean', 'gp_mean', 'intan_mean', 'invt_mean', 'ivst_mean', 'lt_mean', 'ni_mean', 'ppent_mean', 'pstk_mean', 're_mean', 'rect_mean', 'revt_mean', 'seq_mean', 'tstk_mean', 'dvpsp_f_mean', 'dvpsx_f_mean', 'au_mean', 'auop_mean', 'auopic_mean', 'ggroup_mean', 'gind_mean', 'gsector_mean', 'gsubind_mean', 'sic_mean']
Starting training...
fold 0 
Training until validation scores don't improve for 5000 rounds.
[1000]	valid_0's auc: 0.911113
[2000]	valid_0's auc: 0.948038
[3000]	valid_0's auc: 0.959093
[4000]	valid_0's auc: 0.963558
[5000]	valid_0's auc: 0.966769
[6000]	valid_0's auc: 0.969637
[7000]	valid_0

In [27]:
sub_preds.max()

0.5393465258036961

In [28]:
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from sklearn.metrics import accuracy_score
def bestThresshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.001, 0.1, 0.001)):
        tmp[1] = accuracy_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with AUC score: {:.4f}'.format(delta, tmp[2]))
    return delta
threshold = bestThresshold(y,oof_preds)


best threshold is 0.0990 with AUC score: 0.8222


In [29]:
from sklearn.metrics import roc_auc_score
rocauc= roc_auc_score(y, oof_preds)
print("CV score: {:<8.5f}".format(rocauc))

CV score: 0.97688 


In [30]:
submission = pd.read_csv('../input/detecting-accounting-errors/Restate_sampleSubmission.csv')
submission['Restate_Int'] = sub_preds
# submission['Restate_Int'] = submission['Restate_Int'].apply(lambda x : 0 if x < submission['Restate_Int'].quantile(0.98) else 1)

In [31]:
submission.to_csv('submission.csv',index = False)

In [32]:

submission['Restate_Int'].value_counts()

0.170602    5
0.092628    4
0.091532    4
0.142140    3
0.144865    2
0.101916    2
0.008233    2
0.287780    2
0.096311    2
0.121535    2
0.151167    2
0.085337    2
0.090971    2
0.093256    2
0.194316    2
0.042825    2
0.119532    2
0.023680    2
0.065423    2
0.091503    2
0.085435    2
0.013984    2
0.100242    2
0.009087    2
0.266385    2
0.123002    2
0.103952    2
0.090678    2
0.174379    2
0.100733    2
           ..
0.001500    1
0.005150    1
0.003549    1
0.035346    1
0.012002    1
0.022805    1
0.010432    1
0.004210    1
0.011820    1
0.008708    1
0.046246    1
0.046186    1
0.107528    1
0.003265    1
0.012810    1
0.024121    1
0.050653    1
0.005563    1
0.005558    1
0.090707    1
0.016835    1
0.032958    1
0.026266    1
0.097249    1
0.168402    1
0.019276    1
0.013969    1
0.092411    1
0.089577    1
0.022594    1
Name: Restate_Int, Length: 2949, dtype: int64